# 05. 目的変数と説明変数を抽出する

【このノートで実施すること】</br>
　機械学習モデルに読み込ませるための目的変数と説明変数を抽出します。</br>
　抽出したデータは新しいファイルに保存します。</br>
</br>
【入力】</br>
　・data/04_merged_data/df_merged.plk</br>
</br>
【出力】</br>
　・data/05_target_and_features/columns_info.json</br>
　・data/05_target_and_features/df_target.pkl</br>
　・data/05_target_and_features/df_features.pkl</br>
</br>
　※ data/05_target_and_features/columns_info.json は各カラムを目的変数、説明変数、使用しない変数、にどうやって振り分けるかを指定するファイルです。<br>
　　自動で生成されます。詳細は後段で説明しています。
</br>

## 05.1 準備

必要なモジュールをインポートしたり、データを読み込んだりしています。

In [ ]:
# 必要なモジュールをインポート
import pandas as pd
import pickle
import os
import json
from sklearn.model_selection import train_test_split

In [ ]:
# PickleファイルからDataFrameを読み込む
with open('data/04_merged_data/df_merged.pkl', 'rb') as file:
    df_merged = pickle.load(file)

## 05.2 データ抽出用のJsonファイルを生成する

データの中から目的変数に使用するカラムおよび説明変数に使用するカラムを抽出したいです。<br>
しかし、いちいち全部のカラムに対してどれが目的変数に使用するカラムで、どれが説明変数に使用するカラムで、どれが使用しないカラムで、と手作業で指定するのは結構大変です。<br>
そのため、ある程度自動で指定するようにしました。<br>
まず、データに含まれるカラムの一覧を作成し、Json形式で出力します。<br>
その時に各カラムに対して以下のように”usage”という値を持たせます。<br>
<br>
<Strong>"usage"の値の意味</Strong><br>
"target"：目的変数として使用<br>
"feature"：説明変数として使用<br>
"key"：学習に使用しないが残しておきたいカラム<br>
null：学習に使用しない<br>
<br>
この"usage"の値を参照して抽出処理を実行します。<br>
値はJsonファイル生成時に勝手に入るので、手動で設定する手間が省けます。<br>
抽出処理を実行する前に、Jsonファイルを眺めて、"usage"の値を修正したいカラムがあったらそこだけ書き換えればOKです。<br>
<br>
"usage"の値の自動生成のルールは以下のとおりです。<br>
<br>
<Strong>"usage"の値の自動生成ルール：</Strong><br>
引数 target_columns で指定したカラムは"target"が入ります。<br>
引数 key_columns で指定したカラムは"key"が入ります。<br>
型がobjectのカラムはnullが入ります。<br>
型がそれ以外のカラムは"feature"が入ります。<br>

※"KYI_レースキー"カラムは、学習には使用しませんが、残しておきたいカラムだったので"key"に指定しています。<br>
残したい理由は、このカラムを使用してhjcデータを参照したいからです。<br>
hjcデータには払戻金のデータが入っています。機械学習モデルでデータを予測した後の話になるのですが、<br>
もし予測が的中していた時、実際にどれくらいの利益になるのか、を計算するのに使用します。<br>

In [ ]:
# 学習データに含まれるカラムの一覧を作成し、Json形式で出力します。
# 後段の処理で、生成されたJsonファイルの"usage"の値に応じてそのカラムを、
# 目的変数として使用するか、説明変数として使用するか、使用しないか、を判断します。
#
# 〜　"usage"　の値の意味　〜
# "target" 　　-　> 目的変数として使用
# "feature" -　> 説明変数として使用
# "key" 　　　　　　　　-　> 学習に使用しないが残しておきたいカラム
# null 　　　　　　　　 -　> 学習に使用しない
# 　
# "usage"の値の生成ルール：
# 引数 target_columns で指定したカラムは"target"が入ります。
# 引数 key_columns で指定したカラムは"key"が入ります。
# 型がobjectのカラムはnullが入ります。
# 型がそれ以外のカラムは"feature"が入ります。
def save_columns_details_to_json(df, json_filepath, target_columns, key_columns):
    # ファイル名が既に存在する場合、別名で保存する
    # → もし"usage"の値を修正したJsonファイルがある場合、そのファイルが上書きされてしまうのを防ぐためです。
    counter = 1
    base_filename = json_filepath.split('.')[0]
    while os.path.exists(json_filepath):
        filepath = f"{base_filename}_{counter}.json"
        counter += 1
        
    # DataFrameのカラムの詳細をJSON形式で保存
    columns_details = {
        col: {
            'dtype': str(df[col].dtype), 
            'nunique': df[col].nunique(), 
            'usage': 'target' if col in target_columns else 'key' if col in key_columns else (None if df[col].dtype == 'object' else 'feature')
        } for col in df.columns}
    with open(json_filepath, 'w', encoding='utf-8') as file:
        json.dump(columns_details, file, ensure_ascii=False, indent=4)
    print("カラム一覧を保存しました：", json_filepath)

In [ ]:
target_columns = ["SED_馬成績_着順"]
key_columns = ["KYI_レースキー"]
save_columns_details_to_json(df_merged, 'data/05_target_and_features/columns_info.json', target_columns, key_columns)

## 05.3 目的変数と説明変数を抽出する

Jsonファイルに定義した情報に従い、目的変数と説明変数を抽出します。<br>

In [ ]:
# Jsonファイルを読み込む
with open('data/05_target_and_features/columns_info.json', 'r', encoding='utf-8') as file:
    columns_info = json.load(file)

# Jsonファイルに設定した内容を参照し、目的変数データに振り分けるカラムと、説明変数データに振り分けるカラムをまとめる
target_columns = [col for col, details in columns_info.items() if details['usage'] == 'target']
feature_columns = [col for col, details in columns_info.items() if details['usage'] in ('feature', 'key')]

# 元のデータから目的変数データと説明変数データを抽出
df_target = df_merged[target_columns]
df_features = df_merged[feature_columns]

## 05.4 目的変数を加工する

今回、３着以内に入るか、入らないか？を予想したいので、目的変数（SED_馬成績_着順）の値を加工します。<br>
具体的には、３着以内の場合は1,それ以外の場合は0の値を取るカラム”3着以内”を新しく作りました。<br>
ついでに1着の場合は1, それ以外の場合は0の値を取るカラム”１着”も作りました。<br>

In [ ]:
# 1着になるかを予想する場合と、３着以内に入るかを予想する場合のそれぞれについて目的変数を用意してみる

df_target = df_target.copy()

# 1着に入るか否か（1着 = 1, それ以外 = 0）
df_target.loc[:, '1着'] = (df_target['SED_馬成績_着順'] == 1).astype(int)

# 3着に入るか否か（3着以内 = 1, それ以外 = 0）
df_target.loc[:, '3着以内'] = (df_target['SED_馬成績_着順'] <= 3).astype(int)

## 05.5 目的変数と説明変数の確認

生成したデータの中身を確認します。

In [ ]:
# 目的変数データの基本情報を確認
df_features.info()

In [ ]:
# 目的変数データのレコードを確認
df_features.head()

In [ ]:
# 説明変数データの基本情報を確認
df_target.info()

In [ ]:
# 説明変数データのレコードを確認
df_target.head()

## 05.6. データを保存する

生成したデータを保存します。

In [ ]:
# ファイルをpickle形式で保存する
def save_dataframe_to_pickle(dataframe, folder_path, file_name):
    
    print("データをファイルに保存します")

    # フォルダが存在しない場合は作成
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # ファイルのフルパスを構築
    file_path = os.path.join(folder_path, file_name + ".pkl")

    # DataFrame を pickle 形式で保存
    with open(file_path, 'wb') as file:
        pickle.dump(dataframe, file)

    print("保存しました")

In [ ]:
# 目的変数を保存
dataframe = df_target
folder_path = "data/05_target_and_features"
file_name = "df_target"
save_dataframe_to_pickle(dataframe, folder_path, file_name)

In [ ]:
# 説明変数を保存
dataframe = df_features
folder_path = "data/05_target_and_features"
file_name = "df_features"
save_dataframe_to_pickle(dataframe, folder_path, file_name)